In [1]:
import pandas as pd
import glob
pd.options.mode.chained_assignment = None

In [14]:
# search all files inside a specific folder
# *.* means file name with any extension
dir_path = r'*_dbsn.parquet'
data_files = glob.glob(dir_path)
datafiles = []
for f in data_files:
    datafiles.append(pd.read_parquet(f))

In [16]:
data_igm_osm = pd.concat(datafiles)
data_igm_osm = data_igm_osm.fillna(0).reset_index()
del data_igm_osm['index']
columns = data_igm_osm.columns.unique()

In [19]:
data_igm_osm[data_igm_osm.osm_objects > 0].to_csv("data_used_by_igm_from_osm.csv",index=False)